<a href="https://colab.research.google.com/github/salmakhale/Carerha_Data-science/blob/main/Energy_Power_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ###### Part of Practice #####


In [ ]:
# Import Needed Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from datetime import datetime

# Data Connection
# File paths
energy_part1_path = "/content/drive/MyDrive/Data/Final Project/Sites Energy Consumption Part_001.csv"
energy_part2_path = "/content/drive/MyDrive/Data/Final Project/Sites Energy Consumption Part_002.csv"
energy_part3_path = "/content/drive/MyDrive/Data/Final Project/Sites Energy Consumption Part_003.csv"
power_demand_path = "/content/drive/MyDrive/Data/Final Project/Demand.xlsx"

# Read the energy consumption datasets
energy_consumption_df_part1 = pd.read_csv(energy_part1_path)
energy_consumption_df_part2 = pd.read_csv(energy_part2_path)
energy_consumption_df_part3 = pd.read_csv(energy_part3_path)
# Read the power demand dataset
power_demand_df = pd.read_excel(power_demand_path)


# Ensure proper replacement for the year
energy_consumption_df_part1['DateTime'] = energy_consumption_df_part1['DateTime'].apply(lambda x: str(x).replace('13', '2013') if isinstance(x, str) else x)
energy_consumption_df_part2['DateTime'] = energy_consumption_df_part2['DateTime'].apply(lambda x: str(x).replace('13', '2013') if isinstance(x, str) else x)
energy_consumption_df_part3['DateTime'] = energy_consumption_df_part3['DateTime'].apply(lambda x: str(x).replace('13', '2013') if isinstance(x, str) else x)

#00:00:00 13,01,01
energy_consumption_df_part1['DateTime'] = pd.to_datetime(energy_consumption_df_part1['DateTime'],errors='coerce')
energy_consumption_df_part2['DateTime'] = pd.to_datetime(energy_consumption_df_part2['DateTime'], errors='coerce')
energy_consumption_df_part3['DateTime'] = pd.to_datetime(energy_consumption_df_part3['DateTime'], errors='coerce')

# # Identify rows with parsing issues
# invalid_dates = energy_consumption[energy_consumption['DateTime'].isna()]
# print(f"Number of invalid DateTime entries: {len(invalid_dates)}")


# # Fallback to the specific format for rows that were invalid (NaT)
# invalid_rows_part1 = energy_consumption_df_part1['DateTime'].isna()
# invalid_rows_part2 = energy_consumption_df_part2['DateTime'].isna()
# invalid_rows_part3 = energy_consumption_df_part3['DateTime'].isna()

# # Apply the specific format to the invalid rows only
# energy_consumption_df_part1.loc[invalid_rows_part1, 'DateTime'] = pd.to_datetime(energy_consumption_df_part1.loc[invalid_rows_part1, 'DateTime'], format='%H:%M:%S %Y,%m,%d', errors='coerce')
# energy_consumption_df_part2.loc[invalid_rows_part2, 'DateTime'] = pd.to_datetime(energy_consumption_df_part2.loc[invalid_rows_part2, 'DateTime'], format='%H:%M:%S %Y,%m,%d', errors='coerce')
# energy_consumption_df_part3.loc[invalid_rows_part3, 'DateTime'] = pd.to_datetime(energy_consumption_df_part3.loc[invalid_rows_part3, 'DateTime'], format='%H:%M:%S %Y,%m,%d', errors='coerce')

# # Convert DateTime columns to datetime format
# energy_consumption['DateTime'] = pd.to_datetime(energy_consumption['DateTime'], format='%H:%M:%S %Y,%d,%m')
# power_demand['DemandDateTime'] = pd.to_datetime(power_demand['DemandDateTime'], format='%Y-%m-%d %H:%M:%S')

# Combine all three parts of the energy consumption data into one consolidated dataset.
energy_consumption_df = pd.concat([energy_consumption_df_part1, energy_consumption_df_part2, energy_consumption_df_part3], axis=0)



# Convert 'DateTime' column in energy_consumption_df to datetime
energy_consumption_df['DateTime'] = pd.to_datetime(energy_consumption_df['DateTime'], errors='coerce')
# Convert 'DemandDateTime' column in power_demand_df to datetime if not already
power_demand_df['DemandDateTime'] = pd.to_datetime(power_demand_df['DemandDateTime'], errors='coerce')

# power_demand_df.rename(columns={'DemandDateTime': 'DateTime'}, inplace=True)

# # Merge the dataframes
# merged_data = pd.merge(
#     energy_consumption_df,
#     power_demand_df,
#     on='DateTime',
#     how='left'  )


In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380252 entries, 0 to 1380251
Data columns (total 6 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   cell_id                  1380252 non-null  object        
 1   DateTime                 1322783 non-null  datetime64[ns]
 2   KWH/hh (per half hour)   1380252 non-null  float64       
 3   site_id                  1380252 non-null  object        
 4   region                   1380252 non-null  object        
 5   Demand                   1322783 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 63.2+ MB


In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380252 entries, 0 to 1380251
Data columns (total 7 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   cell_id                  1380252 non-null  object        
 1   DateTime                 1322783 non-null  datetime64[ns]
 2   KWH/hh (per half hour)   1380252 non-null  float64       
 3   site_id                  1380252 non-null  object        
 4   region                   1380252 non-null  object        
 5   DemandDateTime           1322783 non-null  datetime64[ns]
 6   Demand                   1322783 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 73.7+ MB


In [ ]:
energy_consumption_df_part1

,cell_id,DateTime,KWH/hh (per half hour),site_id,region
0,MAC000002,"00:00:00 13,01,01",0.219,A,A
1,MAC000002,"00:30:00 13,01,01",0.241,A,A
2,MAC000002,"01:00:00 13,01,01",0.191,A,A
3,MAC000002,"01:30:00 13,01,01",0.235,A,A
4,MAC000002,"02:00:00 13,01,01",0.182,A,A
...,...,...,...,...,...
484382,MAC000035,"21:30:00 13,31,12",1.035,E,A
484383,MAC000035,"22:00:00 13,31,12",0.728,E,A
484384,MAC000035,"22:30:00 13,31,12",0.737,E,A
484385,MAC000035,"23:00:00 13,31,12",0.736,E,A


In [ ]:
energy_consumption_df_part2

,cell_id,DateTime,KWH/hh (per half hour),site_id,region
0,MAC000036,"00:00:00 13,01,01",0.006,E,A
1,MAC000036,"00:30:00 13,01,01",0.017,E,A
2,MAC000036,"01:00:00 13,01,01",0.059,E,A
3,MAC000036,"01:30:00 13,01,01",0.000,E,A
4,MAC000036,"02:00:00 13,01,01",0.000,E,A
...,...,...,...,...,...
444893,MAC000069,"20:00:00 13,12,05",0.061,G,C
444894,MAC000069,"20:30:00 13,12,05",0.090,G,C
444895,MAC000069,"21:00:00 13,12,05",0.069,G,C
444896,MAC000069,"21:30:00 13,12,05",0.084,G,C


In [ ]:
energy_consumption_df_part3

,cell_id,DateTime,KWH/hh (per half hour),site_id,region
0,MAC000069,"22:30:00 13,12,05",0.063,G,C
1,MAC000069,"23:00:00 13,12,05",0.106,G,C
2,MAC000069,"23:30:00 13,12,05",0.029,G,C
3,MAC000069,"00:00:00 13,13,05",0.108,G,C
4,MAC000069,"00:30:00 13,13,05",0.033,G,C
...,...,...,...,...,...
450962,MAC000100,"18:30:00 13,20,12",0.260,N,B
450963,MAC000100,"19:00:00 13,20,12",0.267,N,B
450964,MAC000100,"19:30:00 13,20,12",0.274,N,B
450965,MAC000100,"20:00:00 13,20,12",0.312,N,B


In [ ]:
power_demand_df

,DemandDateTime,Demand
0,2013-01-01 00:00:00,Normal
1,2013-01-01 00:30:00,Normal
2,2013-01-01 01:00:00,Normal
3,2013-01-01 01:30:00,Normal
4,2013-01-01 02:00:00,Normal
...,...,...
17515,2013-12-31 21:30:00,Normal
17516,2013-12-31 22:00:00,Normal
17517,2013-12-31 22:30:00,Normal
17518,2013-12-31 23:00:00,Normal


## Energy Project ###########

In [ ]:
# Import Needed Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from datetime import datetime

# Data Connection
# File paths
energy_part1_path = "/content/drive/MyDrive/Data/Final Project/Sites Energy Consumption Part_001.csv"
energy_part2_path = "/content/drive/MyDrive/Data/Final Project/Sites Energy Consumption Part_002.csv"
energy_part3_path = "/content/drive/MyDrive/Data/Final Project/Sites Energy Consumption Part_003.csv"
power_demand_path = "/content/drive/MyDrive/Data/Final Project/Demand.xlsx"


# Load the datasets
energy_consumption_1 = pd.read_csv(energy_part1_path)
energy_consumption_2 = pd.read_csv(energy_part2_path)
energy_consumption_3 = pd.read_csv(energy_part3_path)
power_demand = pd.read_excel(power_demand_path)

# Combine energy consumption datasets
energy_consumption = pd.concat([energy_consumption_1, energy_consumption_2, energy_consumption_3], ignore_index=True)

# Convert DateTime columns to datetime format
energy_consumption['DateTime'] = pd.to_datetime(energy_consumption['DateTime'], format='%H:%M:%S %y,%d,%m')
power_demand['DemandDateTime'] = pd.to_datetime(power_demand['DemandDateTime'], format='%Y-%m-%d %H:%M:%S')


# Merge energy consumption data with power demand based on the time interval
combined_data = pd.merge(
    energy_consumption,
    power_demand,
    left_on='DateTime',
    right_on='DemandDateTime',
    how='inner'
)

# Check data structure and ensure no missing values
combined_data
print(combined_data.info())

In [ ]:
#Missing
missing_values = combined_data.isnull().sum()
print(missing_values)
#Duplicate
duplicates = combined_data.duplicated().sum()
print(duplicates)
#Describe
print(combined_data.describe())
# #Uniquie
categorical_columns = combined_data.select_dtypes(include='object').columns
for col in categorical_columns:
    print(f"Unique values in {col}: {combined_data[col].nunique()}")

# Check the time range
print(f"Start Date: {combined_data['DateTime'].min()}, End Date: {combined_data['DateTime'].max()}")

# Check which columns are numerical
numerical_columns = combined_data.select_dtypes(include=['float64', 'int64']).columns
print("Numerical Columns for Summing:", numerical_columns)

# Group by date to analyze trends
daily_summary = combined_data.groupby('DateTime')[numerical_columns].sum()
print(daily_summary.head())


In [ ]:
# Uniquie value of column Demand
combined_data['Demand'].unique()
# Convert all text in the 'Demand' column to lowercase and capitalize the first letter
combined_data['Demand'] = combined_data['Demand'].str.strip().str.lower().str.capitalize()
# Define a mapping dictionary
demand_mapping = {
    'low': 'Low',
    'Low': 'Low',
    'High': 'High',
    'high': 'High',
    'HIGH': 'High',
    'normal': 'Normal',
    'Normal': 'Normal',
    'Normall': 'Normal',
    'NORMAL': 'Normal',
}

# Apply the mapping to the 'Demand' column
combined_data['Demand'] = combined_data['Demand'].map(demand_mapping)

# Verify the updated Uniquie value of column Demand
print(combined_data['Demand'].unique())

In [ ]:
# Strip extra spaces from column names
combined_data.columns = combined_data.columns.str.strip()

# Verify the updated column names
print(combined_data.columns)


# Handle Outliers
# Calculate IQR for 'KWH/hh (per half hour)'
Q1 = combined_data['KWH/hh (per half hour)'].quantile(0.25)
Q3 = combined_data['KWH/hh (per half hour)'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(f"Lower Bound: {lower_bound}")
print(f"Upper Bound: {upper_bound}")

# Apply capping to handle outliers
combined_data['KWH/hh (per half hour)'] = combined_data['KWH/hh (per half hour)'].clip(lower=lower_bound, upper=upper_bound)

# Check the summary statistics after capping outliers
print(combined_data.describe())

#check null
missing_data = combined_data.isnull().sum()

# Visualize Trends Over Time:
plt.figure(figsize=(15, 6))
sns.lineplot(x='DateTime', y='KWH/hh (per half hour)', data=combined_data)
plt.title('Energy Consumption Over Time')
plt.show()

In [ ]:
# time-based features from 'DateTime' column
combined_data['hour'] = combined_data['DateTime'].dt.hour
combined_data['day_of_week'] = combined_data['DateTime'].dt.dayofweek
combined_data['month'] = combined_data['DateTime'].dt.month
print(combined_data[['DateTime', 'hour', 'day_of_week', 'month']].head())

# Check the summary statistics
print(combined_data.describe())

In [ ]:
# Exploratory Data Analysis
from scipy.stats import f_oneway ,ttest_ind
import scipy.stats as stats

# Analyze if region significantly affects energy consumption
region_mean_consumption = combined_data.groupby('region')['KWH/hh (per half hour)'].mean()
region_mean_consumption            #averages indicate that Regions A and C have similar energy consumption, while Regions B and D are lower.

# Standardize categorical values
combined_data['region'] = combined_data['region'].str.upper()
combined_data['region'] = combined_data['region'].str.strip()

regions = combined_data['region'].unique()
region_consumptions = [combined_data[combined_data['region'] == region]['KWH/hh (per half hour)'] for region in regions]
# Anova test
anova_result = f_oneway(*region_consumptions)
print(f"F-value: {anova_result[0]}")
print(f"P-value: {anova_result[1]}")        #The p-value is small (0), which means we reject the null hypothesis(energy consumption does vary significantly by region)

# Correlation between Hour and Energy Consumption
hourly_corr = combined_data[['hour', 'KWH/hh (per half hour)']].corr()
print(f"Correlation between Hour and Energy Consumption: {hourly_corr.iloc[0, 1]}")

# T-test Daytime vs. Nighttime energy consumption
daytime_data = combined_data[combined_data['hour'].between(6, 18)]['KWH/hh (per half hour)']
nighttime_data = combined_data[combined_data['hour'].between(18, 6)]['KWH/hh (per half hour)']
t_stat, p_value = stats.ttest_ind(daytime_data, nighttime_data)

In [ ]:
# Plot energy consumption by hour of the day
plt.figure(figsize=(10, 6))
sns.boxplot(x='hour', y='KWH/hh (per half hour)', data=combined_data)
plt.title('Energy Consumption by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Energy Consumption (KWH/hh)')
plt.show()

In [ ]:
# Check the unique hours in the dataset
print(combined_data['hour'].unique())
print(combined_data[['DateTime', 'hour']].head())

# Daytime hours
daytime_data = combined_data[combined_data['hour'].between(6, 18, inclusive='left')]['KWH/hh (per half hour)']

# Nighttime hours (18:00 to 23:59 OR 00:00 to 06:00)
nighttime_data = combined_data[(combined_data['hour'] >= 18) | (combined_data['hour'] < 6)]['KWH/hh (per half hour)']

# Checking counts to ensure there is data for both groups
print(f"Daytime data count: {daytime_data.count()}")
print(f"Nighttime data count: {nighttime_data.count()}")

# Perform T-test if there is data in both subsets
if daytime_data.count() > 1 and nighttime_data.count() > 1:
    t_stat, p_value = stats.ttest_ind(daytime_data, nighttime_data)
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")
else:
    print("Insufficient data for T-test")

In [ ]:
# Energy consumption by region
sns.boxplot(x='region', y='KWH/hh (per half hour)', data=combined_data)
plt.title('Energy Consumption by Region')
plt.show()

In [ ]:
# T-test visualization (Daytime vs Nighttime)
plt.hist(daytime_data, alpha=0.5, label='Daytime', bins=30)
plt.hist(nighttime_data, alpha=0.5, label='Nighttime', bins=30)
plt.legend()
plt.title('Energy Consumption: Daytime vs Nighttime')
plt.show()

In [ ]:
regions = combined_data['region'].unique()

plt.figure(figsize=(12, 6))
# Plotting histograms for each region
for idx, region in enumerate(regions):
    region_data = combined_data[combined_data['region'] == region]['KWH/hh (per half hour)']
    plt.hist(region_data, alpha=0.5, label=region, bins=30)

plt.title('Energy Consumption Distribution by Region')
plt.xlabel('Energy Consumption (KWH/hh)')
plt.ylabel('Frequency')
plt.legend(title='Region')
plt.grid(True)

In [ ]:
# Correlation Heatmap of Energy Consumption and Time-Based Features
correlation_matrix = combined_data[['KWH/hh (per half hour)', 'hour', 'day_of_week', 'month']].corr()
print(correlation_matrix)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='Oranges', fmt='.2f')
plt.title('Correlation Heatmap of Energy Consumption and Time-Based Features')
plt.show()

In [ ]:
# Create a 'time_of_day' column to classify the data
combined_data['time_of_day'] = combined_data['hour'].apply(lambda x: 'Day' if 6 <= x < 18 else 'Night')

plt.figure(figsize=(10, 6))
sns.boxplot(x='time_of_day', y='KWH/hh (per half hour)', data=combined_data, palette='coolwarm')
plt.title('Energy Consumption: Day vs Night')
plt.xlabel('Time of Day')
plt.ylabel('Energy Consumption (KWH/hh)')
plt.grid(True)

In [ ]:
# Convert demand to numerical values
demand_mapping = {'Low': 1, 'Normal': 2, 'High': 3}
combined_data['Demand_numeric'] = combined_data['Demand'].map(demand_mapping)

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Demand_numeric', y='KWH/hh (per half hour)', data=combined_data, alpha=0.5, hue='Demand', palette='viridis')
plt.title('Energy Consumption vs Demand')
plt.xlabel('Demand Level (Low, Normal, High)')
plt.ylabel('Energy Consumption (KWH/hh)')
plt.grid(True)

In [ ]:
import plotly.express as px
# Calculate the average energy consumption by hour
avg_consumption_by_hour = combined_data.groupby('hour')['KWH/hh (per half hour)'].mean().reset_index()

# Create a line plot for average energy consumption by hour
fig = px.line(avg_consumption_by_hour,
              x='hour',
              y='KWH/hh (per half hour)',
              title="Average Energy Consumption by Hour of Day",
              labels={'KWH/hh (per half hour)': 'Average Energy Consumption (KWH/hh)', 'hour': 'Hour of Day'})
fig.show()

In [ ]:
# Create a box plot for Energy Consumption by Region
fig = px.box(combined_data,
             x='region',
             y='KWH/hh (per half hour)',
             points="all",  # Show all points
             title="Energy Consumption Distribution by Region",
             labels={'KWH/hh (per half hour)': 'Energy Consumption (KWH/hh)', 'region': 'Region'})

fig.show()

In [ ]:
# Weekend indicator (1 for weekend, 0 for weekday)
combined_data['is_weekend'] = combined_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Creating a time-of-day feature
def time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

combined_data['time_of_day'] = combined_data['hour'].apply(time_of_day)

# Create a pivot table for energy consumption by day of week and hour
pivot_table = combined_data.pivot_table(index='day_of_week',
                                        columns='hour',
                                        values='KWH/hh (per half hour)',
                                        aggfunc='mean')
pivot_table

In [ ]:
# Group by DateTime to get the daily average consumption
combined_data['Date'] = combined_data['DateTime'].dt.date
daily_avg = combined_data.groupby(['Date', 'is_weekend'])['KWH/hh (per half hour)'].mean().reset_index()

# Plotting the data
plt.figure(figsize=(14, 8))
sns.lineplot(data=daily_avg, x='Date', y='KWH/hh (per half hour)', hue='is_weekend', palette='Set2')
plt.title('Average Energy Consumption: Weekday vs Weekend')
plt.xlabel('Date')
plt.ylabel('Average Energy Consumption (KWH/hh)')
plt.legend(title='Is Weekend', labels=['Weekday', 'Weekend'])
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
# Data Insight & Hypothesis Formulation
# Time-of-Day and Consumption Patterns
hourly_consumption = combined_data.groupby('hour')['KWH/hh (per half hour)'].mean()
print("Average Energy Consumption by Hour of the Day:")
print(hourly_consumption)

# Weekend vs Weekday Consumption
weekend_consumption = combined_data[combined_data['is_weekend'] == 1]['KWH/hh (per half hour)'].mean()
weekday_consumption = combined_data[combined_data['is_weekend'] == 0]['KWH/hh (per half hour)'].mean()
print(f"Weekend Consumption: {weekend_consumption}")
print(f"Weekday Consumption: {weekday_consumption}")

# Correlation between Hour and Energy Consumption
hourly_correlation = combined_data[['hour', 'KWH/hh (per half hour)']].corr().iloc[0, 1]
print(f"Correlation between Hour and Energy Consumption: {hourly_correlation}")

# Visualizing Hourly Consumption
plt.figure(figsize=(10, 6))
sns.lineplot(x=hourly_consumption.index, y=hourly_consumption.values)
plt.title('Average Energy Consumption by Hour of the Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Energy Consumption (KWH/hh)')
plt.grid(True)

In [ ]:
# ANOVA Test for Energy Consumption Across Regions
regions = combined_data['region'].unique()
region_consumptions = [combined_data[combined_data['region'] == region]['KWH/hh (per half hour)'] for region in regions]
anova_result = f_oneway(*region_consumptions)
print(f"ANOVA Test Result for Regions:\nF-value: {anova_result[0]}, P-value: {anova_result[1]}")

# T-test for Daytime vs Nighttime Consumption
daytime_data = combined_data[(combined_data['hour'] >= 6) & (combined_data['hour'] < 18)]['KWH/hh (per half hour)']
nighttime_data = combined_data[(combined_data['hour'] < 6) | (combined_data['hour'] >= 18)]['KWH/hh (per half hour)']

# Perform T-test
t_stat, p_val = ttest_ind(daytime_data, nighttime_data, equal_var=False)
print(f"T-test result between Daytime and Nighttime:\nT-statistic: {t_stat}, P-value: {p_val}")

# Weekend vs Weekday Consumption (Visualization)
plt.figure(figsize=(10, 6))
sns.boxplot(x='is_weekend', y='KWH/hh (per half hour)', data=combined_data)
plt.title('Energy Consumption on Weekends vs Weekdays')
plt.xlabel('Is Weekend')
plt.ylabel('Energy Consumption (KWH/hh)')
plt.xticks([0, 1], ['Weekday', 'Weekend'])
plt.grid(True)
plt.show()

In [ ]:
# Assumptions and discuss limitations
from scipy import stats
regions = combined_data['region'].unique()
region_data = [combined_data[combined_data['region'] == region]['KWH/hh (per half hour)'] for region in regions]

for i, region in enumerate(regions):
    stat, p_value = stats.shapiro(region_data[i])
    print(f'Normality test for {region}: Statistics={stat}, P-value={p_value}')

    # Visualization: Histogram for each region
    plt.hist(region_data[i], bins=30, alpha=0.5, label=f'{region}')

plt.legend(loc='upper right')
plt.title('Energy Consumption Distribution by Region')
plt.show()

stat, p_value = stats.levene(*region_data)
print(f'Levene’s test for homogeneity of variances: Statistics={stat}, P-value={p_value}')

# Separate daytime and nighttime data
daytime_data = combined_data[combined_data['hour'] >= 6]['KWH/hh (per half hour)']
nighttime_data = combined_data[combined_data['hour'] < 6]['KWH/hh (per half hour)']

# test for normality
daytime_stat, daytime_p_value = stats.shapiro(daytime_data)
nighttime_stat, nighttime_p_value = stats.shapiro(nighttime_data)

print(f'Daytime data normality test: Statistics={daytime_stat}, P-value={daytime_p_value}')
print(f'Nighttime data normality test: Statistics={nighttime_stat}, P-value={nighttime_p_value}')

# Levene's test for equal variances
stat, p_value = stats.levene(daytime_data, nighttime_data)
print(f'Levene’s test for equal variances: Statistics={stat}, P-value={p_value}')

# Perform independent t-test for Daytime vs Nighttime
t_stat, t_p_value = stats.ttest_ind(daytime_data, nighttime_data, equal_var=False)  # Welch's t-test when variances are unequal
print(f'T-statistic: {t_stat}, P-value: {t_p_value}')

In [ ]:
# Define a function to categorize seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:  # 9, 10, 11
        return 'Autumn'

# Create a new 'season' column
combined_data['season'] = combined_data['month'].apply(get_season)

# Average energy consumption
seasonal_consumption = combined_data.groupby('season')['KWH/hh (per half hour)'].mean()

# Demand proportions
seasonal_demand = combined_data['season'].value_counts(normalize=True) * 100

# Display results
print("Average Energy Consumption by Season:")
print(seasonal_consumption)

print("\nProportion of Data Points by Season:")
print(seasonal_demand)


In [ ]:
# Bar chart for seasonal energy consumption
seasonal_consumption.plot(kind='bar', color='skyblue', figsize=(8, 5))
plt.title('Average Energy Consumption by Season')
plt.xlabel('Season')
plt.ylabel('Average KWH/hh (per half hour)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Pie chart for seasonal data distribution
fig = go.Figure(data=[go.Pie(
    labels=seasonal_demand.index,
    values=seasonal_demand.values,
    hole=0.4,
    textinfo='percent+label'
)])
fig.update_layout(title='Proportion of Data Points by Season')
fig.show()


In [ ]:
# Threshold Determination
# Assuming combined_data is your DataFrame
kwh_values = combined_data['KWH/hh (per half hour)']

# Calculate the quantiles (percentiles)
q1 = np.percentile(kwh_values, 25)  # 25th percentile
q3 = np.percentile(kwh_values, 75)  # 75th percentile
median = np.median(kwh_values)      # Median (Q2)

# Define consumption categories
low_threshold = q1
high_threshold = q3

# Categorize energy consumption
combined_data['energy_category'] = pd.cut(kwh_values,
                                           bins=[-np.inf, low_threshold, high_threshold, np.inf],
                                           labels=['Low', 'Medium', 'High'])
# Display thresholds
print(f"Low consumption threshold : {low_threshold}")
print(f"High consumption threshold : {high_threshold}")
print(f"Median : {median}")

# distribution of the energy consumption values
plt.hist(kwh_values, bins=50, color='skyblue', alpha=0.7)
plt.axvline(x=low_threshold, color='r', linestyle='--', label='Low Threshold (Q1)')
plt.axvline(x=high_threshold, color='g', linestyle='--', label='High Threshold (Q3)')
plt.title('Energy Consumption Distribution with Thresholds')
plt.xlabel('Energy Consumption (KWh/half-hour)')
plt.ylabel('Frequency')
plt.legend()

In [ ]:
# Cost Savings Calculation
import numpy as np
import pandas as pd

# Demand prices for calculating cost:
# • High (77.21p/kWh)
# • Normal (19.46p/kWh)
# • Low (6.89p/kWh)

# Define threshold values (Q1 and Q3)
low_threshold = 0.065  # Below this is considered low consumption
high_threshold = 0.265  # Above this is considered high consumption

# Define demand prices (in GBP per KWh)
low_demand_price = 0.0689
normal_demand_price = 0.1946
high_demand_price = 0.7721

# Filter the dataset based on consumption thresholds
low_consumption_data = combined_data[combined_data['KWH/hh (per half hour)'] < low_threshold]
normal_consumption_data = combined_data[(combined_data['KWH/hh (per half hour)'] >= low_threshold) &
                                        (combined_data['KWH/hh (per half hour)'] <= high_threshold)]
high_consumption_data = combined_data[combined_data['KWH/hh (per half hour)'] > high_threshold]

# Calculate total energy saved (in KW) by low and high consumption groups
low_consumption_energy = low_consumption_data['KWH/hh (per half hour)'].sum()
normal_consumption_energy = normal_consumption_data['KWH/hh (per half hour)'].sum()
high_consumption_energy = high_consumption_data['KWH/hh (per half hour)'].sum()

# Calculate the potential savings in GBP for low and high consumption groups
low_demand_savings = low_consumption_energy * normal_demand_price
normal_demand_savings = normal_consumption_energy * normal_demand_price
high_demand_savings = high_consumption_energy * normal_demand_price

# Print the savings in KW and GBP
print(f"Total Energy Saved (Low Consumption Users): {low_consumption_energy} KWh")
print(f"Total Energy Saved (Normal Consumption Users): {normal_consumption_energy} KWh")
print(f"Total Energy Saved (High Consumption Users): {high_consumption_energy} KWh")

print(f"Total GBP Savings (Low Consumption Users): {low_demand_savings} GBP")
print(f"Total GBP Savings (Normal Consumption Users): {normal_demand_savings} GBP")
print(f"Total GBP Savings (High Consumption Users): {high_demand_savings} GBP")

In [ ]:
import matplotlib.pyplot as plt

# Define the savings data
energy_savings = {
    "Low Consumption Users": 11578.592,
    "Normal Consumption Users": 100239.608,
    "High Consumption Users": 153243.277
}

gbp_savings = {
    "Low Consumption Users": 2253.1940032,
    "Normal Consumption Users": 19506.6277168,
    "High Consumption Users": 29821.1417042
}

# Plot energy savings bar chart
plt.figure(figsize=(10, 6))
plt.bar(energy_savings.keys(), energy_savings.values(), color='skyblue')
plt.title('Total Energy Savings by Consumption Group')
plt.xlabel('Consumption Group')
plt.ylabel('Total Energy Saved (KWh)')
plt.show()

# Plot GBP savings bar chart
plt.figure(figsize=(10, 6))
plt.bar(gbp_savings.keys(), gbp_savings.values(), color='lightgreen')
plt.title('Total GBP Savings by Consumption Group')
plt.xlabel('Consumption Group')
plt.ylabel('Total GBP Saved')
plt.show()

In [ ]:
import plotly.graph_objects as go

# Define colors for the groups
colors = ['skyblue', 'lightgreen', 'lightcoral']

# pie chart for Total Energy Savings
fig_energy = go.Figure(data=[go.Pie(
    labels=list(energy_savings.keys()),
    values=list(energy_savings.values()),
    hole=0.3, # Makes it a donut chart
    hoverinfo='label+percent',
    textinfo='percent',
    textfont=dict(size=14),
    marker=dict(colors=colors)  # Assign colors
)])

fig_energy.update_layout(
    title="Proportion of Total Energy Savings by Consumption Group",
    showlegend=True
)
fig_energy.show()

# pie chart for Total GBP Savings
fig_gbp = go.Figure(data=[go.Pie(
    labels=list(gbp_savings.keys()),
    values=list(gbp_savings.values()),
    hole=0.3,
    hoverinfo='label+percent',
    textinfo='percent',
    textfont=dict(size=14),
    marker=dict(colors=colors)
)])

fig_gbp.update_layout(
    title="Proportion of Total GBP Savings by Consumption Group",
    showlegend=True
)
fig_gbp.show()